- Name: Isaac Ndirangu Muturi
- Email: ndirangumuturi749@gmail.com

## Module 1 Homework

## Docker & SQL

In this homework we'll prepare the environment 
and practice with Docker and SQL


## Question 1. Knowing docker tags

Run the command to get information on Docker 

```docker --help```

Now run the command to get help on the "docker build" command:

```docker build --help```

Do the same for "docker run".

Which tag has the following text? - *Automatically remove the container when it exits* 

- `--delete`
- `--rc`
- `--rmc`
- `--rm`


In [ ]:
# --rm

#   rm          Remove one or more containers

## Question 2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use ```pip list``` ). 

What is version of the package *wheel* ?

- 0.42.0
- 1.0.0
- 23.0.1
- 58.1.0

In [ ]:
# Run Python 3.9 image in interactive mode with bash as entrypoint
docker run -it --entrypoint /bin/bash python:3.9

# Inside the container, check the version of the 'wheel' package
pip list | grep wheel


# (base) ndirangu749@envy15:~/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/week_1_basics_n_setup/homework$ docker run -it --entrypoint /bin/bash python:3.9
# root@03a78c698bd1:/# pip list
# Package    Version
# ---------- -------
# pip        23.0.1
# setuptools 58.1.0
# wheel      0.40.0


# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from September 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz```

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)

In [ ]:
# refer to upload_data_green_taxi.ipynb 

## Question 3. Count records 

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 15767
- 15612
- 15859
- 89009


In [ ]:
SELECT COUNT(*)
FROM green_taxi_data
WHERE DATE(lpep_pickup_datetime) = '2019-09-18'
  AND DATE(lpep_dropoff_datetime) = '2019-09-18';

# 15612

## Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance?
Use the pick up time for your calculations.

Tip: For every trip on a single day, we only care about the trip with the longest distance. 

- 2019-09-18
- 2019-09-16
- 2019-09-26
- 2019-09-21




In [ ]:
SELECT
  lpep_pickup_datetime::DATE AS pickup_day,
  MAX(trip_distance) AS longest_trip_distance
FROM
  green_taxi_data
GROUP BY
  lpep_pickup_datetime::DATE
ORDER BY
  longest_trip_distance DESC
LIMIT 1;

# 2019-09-26

## Question 5. Three biggest pick up Boroughs

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
 
- "Brooklyn" "Manhattan" "Queens"
- "Bronx" "Brooklyn" "Manhattan"
- "Bronx" "Manhattan" "Queens" 
- "Brooklyn" "Queens" "Staten Island"



In [ ]:
SELECT
  "zones"."Borough",
  SUM("green_taxi_data"."total_amount") AS total_amount_sum
FROM
  "green_taxi_data"
JOIN
  "zones" ON "green_taxi_data"."PULocationID" = "zones"."LocationID"
WHERE
  "green_taxi_data"."lpep_pickup_datetime"::DATE = '2019-09-18'
  AND "green_taxi_data"."PULocationID" IS NOT NULL
GROUP BY
  "zones"."Borough"
HAVING
  SUM("green_taxi_data"."total_amount") > 50000
ORDER BY
  total_amount_sum DESC
LIMIT 3;

# "Brooklyn"
# "Manhattan"
# "Queens"


## Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- JFK Airport
- Long Island City/Queens Plaza

In [ ]:
WITH AstoriaPickups AS (
  SELECT
    "DOLocationID",
    MAX("tip_amount") AS max_tip
  FROM
    "green_taxi_data"
  JOIN
    "zones" ON "green_taxi_data"."PULocationID" = "zones"."LocationID"
  WHERE
    "zones"."Zone" = 'Astoria'
    AND EXTRACT(YEAR FROM "lpep_pickup_datetime") = 2019
    AND EXTRACT(MONTH FROM "lpep_pickup_datetime") = 9
  GROUP BY
    "DOLocationID"
)

SELECT
  "zones"."Zone"
FROM
  "zones"
JOIN
  AstoriaPickups ON "zones"."LocationID" = AstoriaPickups."DOLocationID"
WHERE
  "zones"."Zone" IS NOT NULL
ORDER BY
  AstoriaPickups.max_tip DESC
LIMIT 1;

# JFK Airport

## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. 
Copy the files from the course repo
[here](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/01-docker-terraform/1_terraform_gcp/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.



## Question 7. Creating Resources

After updating the main.tf and variable.tf files run:

```
terraform apply
```

Paste the output of this command into the homework submission form.


(base) ndirangu749@envy15:~/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/week_1_basics_n_setup/homework$ terraform init

Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/google versions matching "5.14.0"...
- Installing hashicorp/google v5.14.0...
- Installed hashicorp/google v5.14.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
(base) ndirangu749@envy15:~/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/week_1_basics_n_setup/homework$ terraform plan

Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the following
symbols:
  + create

Terraform will perform the following actions:

  # google_bigquery_dataset.demo_dataset will be created
  + resource "google_bigquery_dataset" "demo_dataset" {
      + creation_time              = (known after apply)
      + dataset_id                 = "demo_dataset"
      + default_collation          = (known after apply)
      + delete_contents_on_destroy = false
      + effective_labels           = (known after apply)
      + etag                       = (known after apply)
      + id                         = (known after apply)
      + is_case_insensitive        = (known after apply)
      + last_modified_time         = (known after apply)
      + location                   = "europe-north1"
      + max_time_travel_hours      = (known after apply)
      + project                    = "direct-disk-412820"
      + self_link                  = (known after apply)
      + storage_billing_model      = (known after apply)
      + terraform_labels           = (known after apply)
    }

  # google_storage_bucket.demo-bucket will be created
  + resource "google_storage_bucket" "demo-bucket" {
      + effective_labels            = (known after apply)
      + force_destroy               = true
      + id                          = (known after apply)
      + location                    = "EUROPE-NORTH1"
      + name                        = "direct-disk-412820-terra-bucket"
      + project                     = (known after apply)
      + public_access_prevention    = (known after apply)
      + rpo                         = (known after apply)
      + self_link                   = (known after apply)
      + storage_class               = "STANDARD"
      + terraform_labels            = (known after apply)
      + uniform_bucket_level_access = (known after apply)
      + url                         = (known after apply)

      + lifecycle_rule {
          + action {
              + type = "AbortIncompleteMultipartUpload"
            }
          + condition {
              + age                   = 1
              + matches_prefix        = []
              + matches_storage_class = []
              + matches_suffix        = []
              + with_state            = (known after apply)
            }
        }
    }

Plan: 2 to add, 0 to change, 0 to destroy.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Note: You didn't use the -out option to save this plan, so Terraform can't guarantee to take exactly these actions if you run
"terraform apply" now.
(base) ndirangu749@envy15:~/ZOOMCAMP/Data_Engineering_Zoomcamp_2024/week_1_basics_n_setup/homework$ terraform apply

Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the following
symbols:
  + create

Terraform will perform the following actions:

  # google_bigquery_dataset.demo_dataset will be created
  + resource "google_bigquery_dataset" "demo_dataset" {
      + creation_time              = (known after apply)
      + dataset_id                 = "demo_dataset"
      + default_collation          = (known after apply)
      + delete_contents_on_destroy = false
      + effective_labels           = (known after apply)
      + etag                       = (known after apply)
      + id                         = (known after apply)
      + is_case_insensitive        = (known after apply)
      + last_modified_time         = (known after apply)
      + location                   = "europe-north1"
      + max_time_travel_hours      = (known after apply)
      + project                    = "direct-disk-412820"
      + self_link                  = (known after apply)
      + storage_billing_model      = (known after apply)
      + terraform_labels           = (known after apply)
    }

  # google_storage_bucket.demo-bucket will be created
  + resource "google_storage_bucket" "demo-bucket" {
      + effective_labels            = (known after apply)
      + force_destroy               = true
      + id                          = (known after apply)
      + location                    = "EUROPE-NORTH1"
      + name                        = "direct-disk-412820-terra-bucket"
      + project                     = (known after apply)
      + public_access_prevention    = (known after apply)
      + rpo                         = (known after apply)
      + self_link                   = (known after apply)
      + storage_class               = "STANDARD"
      + terraform_labels            = (known after apply)
      + uniform_bucket_level_access = (known after apply)
      + url                         = (known after apply)

      + lifecycle_rule {
          + action {
              + type = "AbortIncompleteMultipartUpload"
            }
          + condition {
              + age                   = 1
              + matches_prefix        = []
              + matches_storage_class = []
              + matches_suffix        = []
              + with_state            = (known after apply)
            }
        }
    }

Plan: 2 to add, 0 to change, 0 to destroy.

Do you want to perform these actions?
  Terraform will perform the actions described above.
  Only 'yes' will be accepted to approve.

  Enter a value: yes

google_bigquery_dataset.demo_dataset: Creating...
google_storage_bucket.demo-bucket: Creating...
google_bigquery_dataset.demo_dataset: Creation complete after 3s [id=projects/direct-disk-412820/datasets/demo_dataset]
google_storage_bucket.demo-bucket: Creation complete after 4s [id=direct-disk-412820-terra-bucket]

Apply complete! Resources: 2 added, 0 changed, 0 destroyed.